# Tutorial. Finite difference methods for stationary elliptic equations

Second-order linear partial differential equations are classified as either [elliptic](https://en.wikipedia.org/wiki/Elliptic_partial_differential_equation), [hyperbolic](https://en.wikipedia.org/wiki/Hyperbolic_partial_differential_equation), or [parabolic](https://en.wikipedia.org/wiki/Parabolic_partial_differential_equation), this naming convention being inspired by the classification of [conic sections](https://en.wikipedia.org/wiki/Conic_section) or [quadratic forms](https://en.wikipedia.org/wiki/Quadratic_form).

The simplest examples of elliptic partial differential equations are the [Laplace equation](https://en.wikipedia.org/wiki/Laplace_equation) (written here in two space dimensions),
$$
-\Delta u(x,y)=-\frac{\partial^2u}{\partial x^2}(x,y)-\frac{\partial^2u}{\partial y^2}(x,y)=0,
$$
and the [Poisson equation](https://en.wikipedia.org/wiki/Poisson_equation),
$$
-\Delta u(x,y)=-\frac{\partial^2u}{\partial x^2}(x,y)-\frac{\partial^2u}{\partial y^2}(x,y)=f(x,y),
$$
where $f$ is a given function.

In this notebook, we are interested in the numerical solution of the Poisson equation by [finite difference methods](https://en.wikipedia.org/wiki/Finite_difference_method).

The <tt>numpy</tt> and <tt>matplotlib</tt> packages will be needed, as well as the `linalg` library of <tt>scipy</tt> (in order to solve linear systems of algebraic equations).

In [ ]:
import numpy as np

# To draw matplotlib plots within this notebook.
%matplotlib inline
import matplotlib.pyplot as plt

import scipy.linalg as linalg

## Exercise 1. The Poisson equation in 1D.
### Part 1. Homogeneous Dirichlet boundary conditions
We first consider the numerical solution, by the finite difference method, of the following boundary value problem for the Poisson equation in one space dimension completed by homogeneous Dirichlet boundary conditions:
$$
\left\{\begin{align*}
&-u''(x)=f(x),\ x\in(a,b),\\
&u(a)=0,\ u(b)=0,
\end{align*}\right.
$$
where $a$ and $b$ are real numbers, such that $a<b$, and $f$ is a given function of class $\mathscr{C}^2$.

The finite difference method for the solution of this problem consists in computing some real numbers $u_0,\dots,u_N$, $N$ being a given non-zero natural integer, solution to a system of algebraic equations or *scheme*, for instance
$$
\left\{\begin{align*}
&-\frac{1}{(\Delta x)^2}(u_{j+1}−2\,u_j+u_{j−1})=f(x_j),\  j=1,\dots,N,\\
&u_0=u_N=0,
\end{align*}\right.
$$
where $\Delta x=\frac{b-a}{N}$ is the (uniform) grid spacing and $x_j=a+j(\Delta x)$, $j=0,\dots,N$, are the gridpoints. The quantities $u_0,\dots,u_N$ are meant to be approximations of the values $u(x_0),\dots,u(x_N)$ of the solution to the problem at the gridpoints $x_0,\dots,x_N$. 

When $N>2$, by setting $U=\begin{pmatrix}u_1\\\vdots\\u_{N-1}\end{pmatrix}$ and $B=\begin{pmatrix}f(x_1)\\\vdots\\f(x_{N-1})\end{pmatrix}$, the last system can be written in matrix form:
$$
\left\{\begin{align*}
&AU=B,\\
&u_0=u_N=0,
\end{align*}\right.
$$
where the matrix
$$
A=\frac{1}{(\Delta x)^2}\begin{pmatrix}2&-1&0&\dots&\dots&0\\-1&2&-1&\ddots&&\vdots\\0&\ddots&\ddots&\ddots&\ddots&\vdots\\\vdots&\ddots&\ddots&\ddots&\ddots&0\\\vdots&&\ddots&-1&2&-1\\0&\dots&\dots&0&-1&2\end{pmatrix}
$$
belongs to $M_{N-1}(\mathbb{R})$ (note here that the unknowns for which the value is readily known, $u_0$ and $u_N$, have been ''eliminated'' from the matrix equation).

In this part, we set $a=0$, $b=1$, and $f(x)=(2\pi)^2\sin(2\pi x)$, so that the solution to the above boundary value problem is
$$
u(x)=\sin(2\pi x).
$$

**Question.** Write a function computing the matrix $A$ defined above, the non-zero natural integer $N$ being an input argument.

In [ ]:
def A(n):
    dx=(b-a)/n
    return (2*np.eye(n-1)-np.diag(np.ones(n-2),1)-np.diag(np.ones(n-2),-1))/dx**2

**Question.** Choosing $N=50$, write a program computing the corresponding approximation to the solution of the Poisson problem using the finite difference method introduced above and the function written in the previous question. The resulting linear system will be solved using a [LU decomposition](https://en.wikipedia.org/wiki/LU_decomposition) of the matrix $A$, which can be done with `linalg.lu_factor` (see the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lu_factor.html)) and `linalg.lu_solve` (see the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lu_solve.html)) functions available in <tt>scipy</tt> (the more efficient [Cholesky decomposition](https://en.wikipedia.org/wiki/Cholesky_decomposition) could also be used, see the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.cho_factor.html), since the matrix is symmetric positive definite). Use this program to plot in the same figure the graph of the analytical solution and its numerical approximation.

In [ ]:
a,b=0,1

def f(x):
    return (2*np.pi)**2*np.sin(2*np.pi*x)

N=50
x=np.linspace(a,b,N+1)

U=np.zeros(N+1)
B=f(x[1:-1])

LU,P=linalg.lu_factor(A(N))
U[1:-1]=linalg.lu_solve((LU,P),B)

plt.figure()
plt.plot(x,U,'*',label='approximation')

def u(x):
    return np.sin(2*np.pi*x)

xx=np.linspace(a,b,1001)
plt.plot(xx,u(xx),label='solution')
plt.xlabel('x')
plt.legend()

We say that the finite difference scheme converges in the supremum norm if the discrete approximation $u_0,\dots,u_N$ is such that:
$$
\lim_{\Delta x\to0}\left(\max_{j=0,\dots,N}|u(x_j)−u_j|\right)=0.
$$

**Question.** For the successive choices $N=2^k$, $k=2,\dots,10$, compute and plot in the same figure the absolute values of the differences between the values of the solution at the gridpoints and their discrete approximations. Comment.

In [ ]:
a,b=0,1

def f(x):
    return (2*np.pi)**2*np.sin(2*np.pi*x)

def u(x):
    return np.sin(2*np.pi*x)

globalerror=[]
steplength=[]

plt.figure()
for k in range(2,11):
    N=2**k
    x=np.linspace(a,b,N+1)

    U=np.zeros(N+1)
    B=f(x[1:-1])

    LU,P=linalg.lu_factor(A(N))
    U[1:-1]=linalg.lu_solve((LU,P),B)

    plt.plot(x,abs(u(x)-U),label='$N=$'+str(N))
    
    globalerror.append(np.max(np.abs(u(x)-U)))
    steplength.append((b-a)/N)

plt.xlabel('x')
plt.legend()
plt.title('error on the grid for different values of $N$')

**Answer.** It is observed that the absolute error decreases with the grid spacing. The method appears to be convergent.

The grid spacing $\Delta x$ being fixed, we denote by
$$
e_{\Delta x}=\max_{j=0,\dots,N}|u(x_j)−u_j|
$$
the global error of the scheme associated with the spacing $\Delta x$. The convergence of the method then implies that
$$
\lim_{\Delta x\to 0}e_{\Delta x}=0.
$$

For the above scheme, it can be shown that the convergence rate is at least of order two, that is, there exists a positive constant $C$, which does not depend on $\Delta x$, such that
$$
e_{\Delta x}\leq C\,(\Delta x)^2.
$$

**Question.** Compute, for each value of $\Delta x$ associated to a value of $N=2^k$, $k=2,\dots,10$, the global error of the method (this can be done by adding a few lines of code in the previously written program). Plot, in a single figure and using logarithmic scales, the global error as a function of the grid spacing, as well a straight line with slope equal to $2$ for comparison purposes. Comment.

In [ ]:
plt.figure()
plt.loglog(steplength,globalerror,'*')
plt.loglog(np.array(steplength),np.array(steplength)**2,label='slope 2')
plt.xlabel('$\Delta x$')
plt.ylabel('global error')
plt.legend()

**Answer.** The rate of convergence appears to agree with the theory. The effective order of the method can be computed using a linear regression.

In [ ]:
slope=np.polyfit(np.log(steplength),np.log(globalerror),1)
print('The effective order of the method is',slope[0],'.')

### Part 2. Non-homogeneous Dirichlet boundary conditions
We now consider the Poisson equation completed with non-homogeneous Dirichlet boundary conditions:
$$
\left\{\begin{align*}
&-u''(x)=f(x),\ x\in(a,b),\\
&u(a)=\alpha,\ u(b)=\beta,
\end{align*}\right.
$$
where $\alpha$ and $\beta$ are two given non-zero real numbers. To approximate the solution to this boundary-value problem, the previous finite difference scheme is used.

**Question.** Explain what are the modifications to the linear algebraic system to solve entailed by the changes in the boundary conditions of the problem compared to the homogeneous case.

**Answer.** It follows from the substitution of the boundary data in the scheme that the right-hand side of the matrix system is now
$$
B=\begin{pmatrix}f(x_1)+\frac{\alpha}{(\Delta x)^2}\\f(x_2)\\\vdots\\f(x_{N-2})\\f(x_{N-1})+\frac{\beta}{(\Delta x)^2}\end{pmatrix}.
$$

**Question.** On the model of the previous one, write a program computing an approximation of the solution to the following Poisson problem 
$$
\left\{\begin{align*}
&-u''(x)=-\frac{2}{(1+x)^3},\ x\in(1,2),\\
&u(1)=\frac{1}{2},\ u(2)=\frac{1}{3},
\end{align*}\right.
$$
which is given by
$$
u(x)=\frac{1}{1+x}.
$$
Plot in the same figure the graph of the solution and its numerical approximation, and illustrate the convergence of the scheme in another figure. Comment.

In [ ]:
a,b=1,2

def f(x):
    return -2./(x+1.)**3

alpha,beta=0.5,1./3.

N=50
x=np.linspace(a,b,N+1)
dx=(b-a)/N

U=np.zeros(N+1)
B=f(x[1:-1])
B[0]+=alpha/dx**2
B[-1]+=beta/dx**2

U[0]=alpha
LU,P=linalg.lu_factor(A(N))
U[1:-1]=linalg.lu_solve((LU,P),B)
U[-1]=beta

fig,axs=plt.subplots(1,2,figsize=(10,4))
axs[0].plot(x,U,'*',label='approximation')

def u(x):
    return 1./(1.+x)

xx=np.linspace(a,b,1001)
axs[0].plot(xx,u(xx),label='solution')
axs[0].set_xlabel('x')
axs[0].legend()

globalerror=[]
steplength=[]

for k in range(2,11):
    N=2**k
    x=np.linspace(a,b,N+1)
    dx=(b-a)/N

    U=np.zeros(N+1)
    B=f(x[1:-1])
    B[0]+=alpha/dx**2
    B[-1]+=beta/dx**2

    U[0]=alpha
    LU,P=linalg.lu_factor(A(N))
    U[1:-1]=linalg.lu_solve((LU,P),B)
    U[-1]=beta

    globalerror.append(np.max(np.abs(u(x)-U)))
    steplength.append(dx)

axs[1].loglog(steplength,globalerror,'*')
axs[1].loglog(np.array(steplength),np.array(steplength)**2,label='slope 2')
axs[1].set_xlabel('$\Delta x$')
axs[1].set_ylabel('global error')
axs[1].legend()

fig.tight_layout()

**Answer.** Here again, the rate of convergence observed in practice agrees with the theory. The effective order of the method can be computed using a linear regression.

In [ ]:
slope=np.polyfit(np.log(steplength),np.log(globalerror),1)
print('The effective order of the method is',slope[0])

We next consider the following Dirichlet-Poisson problem:
$$
\left\{\begin{align*}
&-u''(x)=1,\ x\in(1,2),\\
&u(1)=1,\ u(2)=2.
\end{align*}\right.
$$

**Question.** Determine the solution to this problem.

**Answer.** Integrating twice and using the boundary conditions, one finds
$$u(x)=-\frac{x^2}{2}+\frac{5}{2}x-1.$$

**Question.** Write a program computing an approximation to the solution of the problem based on the finite difference scheme previously considered. Plot in the same figure the graph of the solution and its numerical approximation, and illustrate the convergence of the method in another figure. Comment.

In [ ]:
a,b=1,2

def f(x):
    return np.ones(x.shape)

alpha,beta=1.,2.

N=50
x=np.linspace(a,b,N+1)
dx=(b-a)/N

U=np.zeros(N+1)
B=f(x[1:-1])
B[0]+=alpha/dx**2
B[-1]+=beta/dx**2

U[0]=alpha
LU,P=linalg.lu_factor(A(N))
U[1:-1]=linalg.lu_solve((LU,P),B)
U[-1]=beta

fig,axs=plt.subplots(1,2,figsize=(10,4))
axs[0].plot(x,U,'*',label='approximation')

def u(x):
    return -0.5*x**2+2.5*x-1.

xx=np.linspace(a,b,1001)
axs[0].plot(xx,u(xx),label='solution')
axs[0].set_xlabel('x')
axs[0].legend()

globalerror=[]
steplength=[]

for k in range(2,11):
    N=2**k
    x=np.linspace(a,b,N+1)
    dx=(b-a)/N

    U=np.zeros(N+1)
    B=f(x[1:-1])
    B[0]+=alpha/dx**2
    B[-1]+=beta/dx**2
    U[0]=alpha

    LU,P=linalg.lu_factor(A(N))
    U[1:-1]=linalg.lu_solve((LU,P),B)
    U[-1]=beta

    globalerror.append(np.max(np.abs(u(x)-U)))
    steplength.append(dx)

axs[1].loglog(steplength,globalerror,'*')
axs[1].loglog(np.array(steplength),np.array(steplength)**2,label='slope 2')
axs[1].set_xlabel('$\Delta x$')
axs[1].set_ylabel('global error')
axs[1].legend()

fig.tight_layout()

**Answer.** We observe that he global error for the last problem is much smaller than in the previous examples. We can check that it is close to [machine precision](https://en.wikipedia.org/wiki/Machine_epsilon) for moderate values of $N$.

In [ ]:
for i in range(9):
    print('global error for Delta x=',steplength[i],' :',globalerror[i])

**Question.** Explain why this case is different from the previous ones.

**Answer.** We have seen that the solution is a polynomial function of degree $2$, for which any approximation by the above finite difference scheme is theoretically exact. The fact that the error increases as $\Delta x$ tends to $0$ is due to the accumulation of rounding errors in the computations done in [floating-point arithmetic](https://en.wikipedia.org/wiki/Floating-point_arithmetic), the [condition number](https://en.wikipedia.org/wiki/Condition_number) of the matrix $A$ being inversely proportional to $\Delta x$.

In [ ]:
for k in range(2,11):
    N=2**k
    print('2-norm condition number of the matrix A for Delta x=',(b-a)/N,' :',np.linalg.cond(A(N)))

### Part 3. Homogeneous Neumann boundary conditions

We finally consider the following boundary-value problem:
$$
\left\{\begin{align*}
&-u''(x)+u(x)=f(x),\ x\in(a,b),\\
&u'(a)=0,\ u'(b)=0,
\end{align*}\right.
$$
in which the boundary conditions are homogeneous Neumann ones.

### A first approach.
We keep using the discretisation based on an uniform grid of the interval $(a,b)$ employed in the first parts of the exercise. To approximate the values of the first derivative at the endpoints in this setting, one can naturally employ the following, respectively forward and backward, finite difference formulas:
$$
u'(a)\simeq\frac{u(a+\Delta x)−u(a)}{\Delta x},\ u'(b)\simeq\frac{u(b)−u(b-\Delta x)}{\Delta x},
$$
leading to the scheme:
$$
\left\{\begin{align*}
&\frac{1}{\Delta x^2}(u_{j-1}-2\,u_j+u_{j+1})+u_j=f(x_j),\ j=1,\dots,N-1,\\
&\frac{u_1−u_0}{\Delta x}=0,\ \frac{u_N−u_{N-1}}{\Delta x}=0.
\end{align*}\right.
$$

This linar system of algebraic equations can be be written in the matrix form as
$$
\left\{\begin{align*}
&(\tilde{A}+I_{N-1})U=B\\
&u_0=u_1,\ u_N=u_{N-1},
\end{align*}\right.
$$
where $\tilde{A}$ is a matrix of order $N-1$, $I_{N-1}$ is the identity matrix of order $N-1$, $U=\begin{pmatrix}u_1\\\vdots\\u_{N-1}\end{pmatrix}$, and $B=\begin{pmatrix}f(x_1)\\\vdots\\f(x_{N-1})\end{pmatrix}$.

**Question.** Determine the matrix $\tilde{A}$ and write a function which computes it, the non-zero natural integer $N$ being an input argument.

**Answer.** It follows from the approximation of the boundary conditions that one has $u_0=u_1$ and $u_N=u_{N-1}$, so that
$$
\tilde{A}=\frac{1}{(\Delta x)^2}\begin{pmatrix}1&-1&0&\dots&\dots&0\\-1&2&-1&\ddots&&\vdots\\0&\ddots&\ddots&\ddots&\ddots&\vdots\\\vdots&\ddots&\ddots&\ddots&\ddots&0\\\vdots&&\ddots&-1&2&-1\\0&\dots&\dots&0&-1&1\end{pmatrix}.
$$

In [ ]:
def tildeA(n):
    dx=(b-a)/n
    AA=2*np.eye(n-1)-np.diag(np.ones(n-2),1)-np.diag(np.ones(n-2),-1)
    AA[0,0]=1
    AA[-1,-1]=1
    return AA/dx**2

We now set $a=0$, $b=1$, and $f(x)=((2\pi)^2+1)\cos(2\pi x)$, so that the solution of the above problem is given by
$$
u(x)=\cos(2\pi x).
$$

**Question.** Write a program computing an approximation to the solution of the problem based on the above finite difference scheme. For $N=100$, plot in the same figure the graph of the solution and its numerical approximation.

In [ ]:
a,b=0,1

def f(x):
    return ((2*np.pi)**2+1)*np.cos(2*np.pi*x)

N=100
x=np.linspace(a,b,N+1)
dx=(b-a)/N

U=np.zeros(N+1)
B=f(x[1:-1])

U[0]=alpha
LU,P=linalg.lu_factor(tildeA(N)+np.eye(N-1))
U[1:-1]=linalg.lu_solve((LU,P),B)
U[0]=U[1]
U[-1]=U[-2]

plt.plot(x,U,'*',label='approximation')

def u(x):
    return np.cos(2*np.pi*x)

xx=np.linspace(a,b,1001)
plt.plot(xx,u(xx),label='solution')
plt.xlabel('x')
plt.legend()

**Question.** Illustrate the convergence of the method in a figure. What is the effective rate of convergence of the method?

In [ ]:
globalerror=[]
steplength=[]

for k in range(2,11):
    N=2**k
    x=np.linspace(a,b,N+1)
    dx=(b-a)/N

    U=np.zeros(N+1)
    B=f(x[1:-1])

    LU,P=linalg.lu_factor(tildeA(N)+np.eye(N-1))
    U[1:-1]=linalg.lu_solve((LU,P),B)
    U[0]=U[1]
    U[-1]=U[-2]

    globalerror.append(np.max(np.abs(u(x)-U)))
    steplength.append(dx)

plt.loglog(steplength,globalerror,'*')
plt.loglog(np.array(steplength),np.array(steplength),label='slope 1')
plt.loglog(np.array(steplength),np.array(steplength)**2,label='slope 2')
plt.xlabel('$\Delta x$')
plt.ylabel('global error')
plt.legend()

**Answer.** The observed rate of convergence is not equal to $2$, but most likely close to $1$. This is confirmed by the computation of the effective order and due to use of a first-order finite difference for the approximation of the first derivatives appearing in the Neumann boundary conditions.

In [ ]:
slope=np.polyfit(np.log(steplength),np.log(globalerror),1)
print('The effective order of the method is',slope[0])

### Second-order approximation of the boundary conditions.

**Question.** Using the expansion $u(a+\Delta x)=u(a)+\Delta xu'(a)+\frac{(\Delta x)^2}{2}u''(a)+O((\Delta x)^3)$, rewritten as
$$
u'(a)=\frac{1}{\Delta x}\left(u(a+\Delta x)−u(a)-\frac{(\Delta x)^2}{2}u''(a)\right)+O((\Delta x)^2)=\frac{1}{\Delta x}(u(a+\Delta x)−u(a))+\frac{\Delta x}{2}(f(a)−u(a))+O((\Delta x)^2),
$$
devise a second-order approximation for the boundary condition $u'(a)=0$. Do the same for the remaining boundary condition and write the resulting scheme into an explicit matrix form.

**Answer.** We infer the following approximation from the truncated expansion:
$$
u_0=\frac{2u_1+(\Delta x)^2f(a)}{2+(\Delta x)^2}.
$$
For the boundary condition at the other endpoint, one gets similarly
$$
u_N=\frac{2u_{N-1}+(\Delta x)^2f(b)}{2+(\Delta x)^2},
$$
resulting in the matrix
$$
\tilde{A}=\frac{1}{(\Delta x)^2}\begin{pmatrix}2-\frac{2}{2+(\Delta x)^2}&-1&0&\dots&\dots&0\\-1&2&-1&\ddots&&\vdots\\0&\ddots&\ddots&\ddots&\ddots&\vdots\\\vdots&\ddots&\ddots&\ddots&\ddots&0\\\vdots&&\ddots&-1&2&-1\\0&\dots&\dots&0&-1&2-\frac{2}{2+(\Delta x)^2}\end{pmatrix}
$$
and the right-hand side
$$
B=\begin{pmatrix}\frac{f(a)}{2++(\Delta x)^2}\\f(x_1)\\\vdots\\f(x_{N-1})\\\frac{f(b)}{2++(\Delta x)^2}\end{pmatrix}
$$
for the matrix form of the scheme.

**Question.** Write a program computing an approximation to the solution of the problem based on this new scheme and verify that it is effectively of order $2$.

In [ ]:
a,b=0,1

def f(x):
    return ((2*np.pi)**2+1)*np.cos(2*np.pi*x)

def u(x):
    return np.cos(2*np.pi*x)

def tildeA(n):
    dx=(b-a)/n
    AA=2*np.eye(n-1)-np.diag(np.ones(n-2),1)-np.diag(np.ones(n-2),-1)
    AA[0,0]+=-2./(2.+dx**2)
    AA[-1,-1]+=-2./(2.+dx**2)
    return AA/dx**2

globalerror=[]
steplength=[]

for k in range(2,11):
    N=2**k
    x=np.linspace(a,b,N+1)
    dx=(b-a)/N

    U=np.zeros(N+1)
    B=f(x[1:-1])
    B[0]+=f(a)/(2.+dx**2)
    B[-1]+=f(b)/(2.+dx**2)

    LU,P=linalg.lu_factor(tildeA(N)+np.eye(N-1))
    U[1:-1]=linalg.lu_solve((LU,P),B)
    U[0]=2.*U[1]/(2.+dx**2)+(dx**2)*f(a)/(2.+dx**2)
    U[-1]=2.*U[-2]/(2+dx**2)+(dx**2)*f(b)/(2.+dx**2)

    globalerror.append(np.max(np.abs(u(x)-U)))
    steplength.append(dx)

plt.figure()
plt.loglog(steplength,globalerror,'*')
plt.loglog(np.array(steplength),np.array(steplength)**2,label='slope 2')
plt.xlabel('$\Delta x$')
plt.ylabel('global error')
plt.legend()

slope=np.polyfit(np.log(steplength),np.log(globalerror),1)
print('The effective order of the method is',slope[0])

## Exercise 2. The Poisson equation in 2D.

We are interested in the numerical solution by a finite difference scheme of a boundary-value problem for the Poisson equation, completed by an homogeneous Dirichlet boundary condition, in a square domain of $\mathbb{R}^2$:
$$
\left\{\begin{align*}
&-\Delta u=f\text{ in }\Omega,\\
&u=0\text{ on }\partial\Omega,
\end{align*}\right.
$$
where $\Omega=(a,b)\times(a,b)\subset\mathbb{R}^2$ ans $f:\Omega\to\mathbb{R}$ is a given function of class $\mathscr{C}^2$. 

To do so, the domain $\Omega$ is replaced by a discretisation grid with uniform spacing $\Delta x=\Delta y=\frac{b−a}{N}=h$ in both space directions, with $N$ a given non-zero natural integer, by setting
$$
x_i=a+ih,\ i=1,\dots,N,\text{ and }y_j=a+jh,\ j=0,\dots,N.
$$
Note here that the gridpoints $(x_i,y_j)$ with indices $i$ and $j$ in $1,\dots,N-1$ are interior points of the domain.

The Laplace operator is then approximated by a finite difference with a five-point stencil at each of the interior gridpoints, that is
$$
\Delta u(x_i,y_j)\simeq\frac{1}{h^2}\left(u(x_{i+1},y_j)+u(x_{i−1},y_j)+u(x_i,y_{j+1})+u(x_i,y_{j−1})-4u(x_i,y_j)\right),\ i=1,\dots,N-1,\ j=1,\dots,N-1.
$$
the values of the approximation at the gridpoints on the boundary $\partial\Omega$ being equal to $0$.

In order to write the linear system resulting from this scheme, it is necessary to number the gridpoints. We use the following convention: $P_1=(x_1,y_1), P_2=(x_2,y_1),\dots, P_{N-1}=(x_{N-1},y_1), P_N=(x_1,y_2),\dots, P_{(N-1)^2}=(x_{N-1},y_{N-1})$. We then denote by $u_i$ the approximate value of the solution at point $P_i$, $i=1,\dots,(N-1)^2$.

**Question.** Write the algebraic equation satisfied by the approximation of the solution at an interior gridpoint of the domain $\Omega$, using only the index of the gridpoint numbering. Check that the linear system of $(N-1)^2$ equations can be written in the form $AU=B$, where $A$ is a symmetric matrix of order $(N-1)^2\times(N-1)^2$ which as the following block partition
$$
A=-\frac{1}{h^2}\begin{pmatrix}C&I_{N-1}\\I_{N-1}&C&\ddots\\&\ddots&\ddots&\ddots\\&&\ddots&C&I_{N-1}\\&&&I_{N-1}&C\end{pmatrix}
$$
with $I_{N-1}$ the identity matrix of order $N-1$ and $C$ the tridiagonal matrix of order $N-1$ defined by
$$
C=\begin{pmatrix}-4&1&0&\dots&\dots&0\\1&-4&-1&\ddots&&\vdots\\0&\ddots&\ddots&\ddots&\ddots&\vdots\\\vdots&\ddots&\ddots&\ddots&\ddots&0\\\vdots&&\ddots&1&-4&1\\0&\dots&\dots&0&1&-4\end{pmatrix},
$$
the other blocks being zero.

**Answer.** Denoting by $f_i$ the value of the function $f$ at an interior gridpoint $P_i$, $i=1,\dots,(N-1)^2$, we have (ignoring the particularities when $P_i$ is adjacent to a gridpoint on the boundary)
$$
-\frac{1}{h^2}\left(u_{i+1}+u_{i−1}+u_{i+N-1}+u_{i-N+1}-4u_i)\right)=f_i,
$$
which leads to a structure with only five non-zero diagonals for the matrix $A$.

**Question.** Write a function which computes the matrix $A$, the non-zero natural integer $N$ being an input argument. To do so, one may use the [Kronecker product](https://en.wikipedia.org/wiki/Kronecker_product) of matrices via the function `kron` of <tt>numpy</tt> (see the [documentation](https://numpy.org/doc/stable/reference/generated/numpy.kron.html), or try for instance the commands `A1=np.eye(3)`, `M1=np.ones((2,2))`, `A=np.kron(A1,M1))` and analyse the result).

In [ ]:
def A(n):
    C=-4*np.eye(n-1)+np.diag(np.ones(n-2),1)+np.diag(np.ones(n-2),-1)
    AA=np.kron(np.eye(n-1),C)+np.kron(np.diag(np.ones(n-2),1),np.eye(n-1))+np.kron(np.diag(np.ones(n-2),-1),np.eye(n-1))
    return AA

We now set $a=0$, $b=1$, and $f(x,y)=2\pi^2\sin(\pi x)\sin(\pi y)$, so that the solution of the above problem is given by
$$
u(x,y)=\sin(\pi x)\sin(\pi y).
$$

**Question.** Write a program computing an approximation to the solution of the above Poisson--Dirichlet problem and plot it with the solution using either the `matplotlib.pyplot.pcolor` function (see the [documentation](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.pcolor.html)) or as a [3D surface](https://matplotlib.org/stable/gallery/mplot3d/surface3d.html) with <tt>matplotlib</tt>. Check the convergence of the method using a few different values of $N$.

**Beware of the computational complexity of LU decomposition algorithm with respect to the order of the matrix of the linear system**: computation time could be long!

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

a,b=0,1

def u(x,y):
    return np.sin(np.pi*x)*np.sin(np.pi*y)

def f(x,y):
    return 2*((np.pi)**2)*np.sin(np.pi*x)*np.sin(np.pi*y)

N=39

h=(b-a)/N

x=np.linspace(a,b,N+1)
y=np.linspace(a,b,N+1)

X,Y=np.meshgrid(x,y)
Xint,Yint=np.meshgrid(x[1:-1],y[1:-1])

FF=f(Xint,Yint)
B=-(h**2)*FF.reshape((N-1)**2,1)

U=np.zeros((N+1,N+1))
LU,P=linalg.lu_factor(A(N))
UU=linalg.lu_solve((LU,P),B)
U[1:-1,1:-1]=UU.reshape(N-1,N-1)

Z=u(X,Y)

fig=plt.figure(figsize=plt.figaspect(0.4))

ax=fig.add_subplot(1,3,1,projection='3d')
ax.plot_surface(X,Y,Z,cmap=cm.coolwarm,linewidth=0,antialiased=False)
ax.set_title('solution')

ax=fig.add_subplot(1,3,2,projection='3d')
ax.plot_surface(X,Y,U,cmap=cm.coolwarm,linewidth=0, antialiased=False)
ax.set_title('approximation')

ax=fig.add_subplot(1,3,3,projection='3d')
ax.plot_surface(X,Y,abs(U-Z),cmap=cm.coolwarm,linewidth=0, antialiased=False)
ax.set_title('error')

globalerror=[]
steplength=[]
    
for k in range(2,6):
    N=2**k
    h=(b-a)/N
    
    x=np.linspace(a,b,N+1)
    y=np.linspace(a,b,N+1)

    X,Y=np.meshgrid(x,y)
    Xint,Yint=np.meshgrid(x[1:-1],y[1:-1])

    FF=f(Xint,Yint)
    F=-(h**2)*FF.reshape((N-1)**2,1)

    U=np.zeros((N+1,N+1))
    UU=np.linalg.solve(A(N),F)
    U[1:-1,1:-1]=UU.reshape(N-1,N-1)

    Z=u(X,Y)
    
    globalerror.append(np.max(np.abs(U-Z)))
    steplength.append(h)
   
plt.figure()
plt.loglog(steplength,globalerror,'*')
plt.loglog(np.array(steplength),np.array(steplength)**2,label='slope 2')
plt.xlabel('$h$')
plt.ylabel('global error')
plt.legend()

**Question.** How should the memory cost and the runtime of this program evolve with respect to $N$?

**Answer.** The bottleneck of the program is tied to the numerical solution of the linear system of algebraic equations. If the sparsity of $A$ is not taken into account and the matrix is fully stored, the memory cost should behave like $O(N^4)$ for $N$ large enough (the matrix has $n^2$ entries and $n=(N-1)^2$). Likewise, the complexity of the LU decomposition should behave like $O(N^6)$ for $N$ large enough (the decomposition cost is about $\frac{2}{3}n^3$ arithmetic operations for $n$ large enough and $n=(N-1)^2$), and so should the runtime.

One can measure the runtime of an algorithm by using successive calls to the with the with the `time` function in the `time` library of <tt>python</tt> (see the [documentation](https://docs.python.org/3/library/time.html#time.time)) as follows:
```
import time

start_time = time.time()

# algorithm whose runtime is to be measured

elapsed_time = time.time() - start_time
```

**Question.** For a reasonable range of values of $N$, plot the elapsed time for the method as function of $N$ using a log-log scale. Is the expected asymptotic trend observed?

In [ ]:
import time

a,b=0,1

def f(x,y):
    return 2*((np.pi)**2)*np.sin(np.pi*x)*np.sin(np.pi*y)

elapsed_time=[]
NN=[]

for k in range(2,8):
    start_time=time.time()
    N=2**k
    h=(b-a)/N

    x=np.linspace(a,b,N+1)
    y=np.linspace(a,b,N+1)

    X,Y=np.meshgrid(x,y)
    Xint,Yint=np.meshgrid(x[1:-1],y[1:-1])

    FF=f(Xint,Yint)
    B=-(h**2)*FF.reshape((N-1)**2,1)
    
    U=np.zeros((N+1,N+1))
    LU,P=linalg.lu_factor(A(N))
    UU=linalg.lu_solve((LU,P),B)
    U[1:-1,1:-1]=UU.reshape(N-1,N-1)

    elapsed_time.append(time.time()-start_time)
    NN.append(N)
    print('Runtime for N=',N,':',elapsed_time[-1],'s.')

plt.loglog(NN,elapsed_time)
plt.loglog(np.array(NN),np.array(NN)**6,label='slope 6')
plt.xlabel('$N$')
plt.ylabel('runtime in seconds')
plt.legend()

**Answer.** The expected trend is observed for larger values of $N$.

The matrix $A$ being a banded matrix with only four non-zero diagonals, the number of non-zero coefficients is highly negligible compared to the number of zero coefficients when $N$ is large. Such a matrix called [sparse](https://en.wikipedia.org/wiki/Sparse_matrix) and it is beneficial, if not necessary, to use specialised algorithms and data structures that take advantage of this property in practice.

The `sparse` library in <tt>scipy</tt> (see the [documentation](https://docs.scipy.org/doc/scipy/reference/sparse.html)) provides functions to properly define and manipulate sparse matrices.

**Question.** Using the <tt>sparse</tt> library, write a function which computes the matrix $A$ and stores it in a Compressed Sparse Column matrix (CSC) data structure, the non-zero natural integer $N$ being an input argument.

In [ ]:
import scipy.sparse as sparse
def spA(n):
    C=-4*sparse.eye(n-1,format='csc')+sparse.diags([np.ones(n-2),np.ones(n-2)],[-1,1],format='csc')
    AA=sparse.kron(sparse.eye(n-1,format='csc'),C,format='csc')+sparse.kron(sparse.diags(np.ones(n-2),1,format='csc'),sparse.eye(n-1,format='csc'),format='csc')+sparse.kron(sparse.diags(np.ones(n-2),-1,format='csc'),sparse.eye(n-1,format='csc'),format='csc')
    return AA

**Question.** On the model of the previous program, write a program for solving the Poisson problem using functions taking into account the sparse structure of the matrix (see the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.splu.html) for the `sparse.linalg.splu` function). Do you observe a difference in the runtime scaling?

In [ ]:
from scipy.sparse.linalg import splu

import time

a,b=0,1

def f(x,y):
    return 2*((np.pi)**2)*np.sin(np.pi*x)*np.sin(np.pi*y)

elapsed_time=[]
NN=[]

for k in range(2,11):
    start_time=time.time()
    N=2**k
    h=(b-a)/N

    x=np.linspace(a,b,N+1)
    y=np.linspace(a,b,N+1)

    X,Y=np.meshgrid(x,y)
    Xint,Yint=np.meshgrid(x[1:-1],y[1:-1])

    FF=f(Xint,Yint)
    B=-(h**2)*FF.reshape((N-1)**2,1)
    
    U=np.zeros((N+1,N+1))
    LU=splu(spA(N))
    UU=LU.solve(B)
    U[1:-1,1:-1]=UU.reshape(N-1,N-1)

    elapsed_time.append(time.time()-start_time)
    NN.append(N)
    print('Runtime for N=',N,':',elapsed_time[-1],'s.')

plt.loglog(NN,elapsed_time)
plt.loglog(np.array(NN),np.array(NN)**2,label='slope 2')
plt.xlabel('$N$')
plt.ylabel('runtime in seconds')
plt.legend()

**Answer.** We observe that the runtime grows linearly with order of the matrix (that is like $O(N^2)$). Larger values of $N$ can be used with thanks to the highly reduced memory storage.